In [ ]:
#Importing library
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.utils import *
from keras.initializers import *
import tensorflow as tf
import time, random
import hashlib
from sklearn.model_selection import train_test_split

In [ ]:
f = open("eng-french.txt", "r")  #Load file in any mode that's able to read, ie r, r+, w+ etc
len(f.readlines())               #to get length



149863

In [ ]:

# remove duplicate
output_file_path = "EN-FR.txt"
input_file_path = "eng-french.txt"
completed_lines_hash = set()
output_file = open(output_file_path, "w")
for line in open(input_file_path, "r"):

  hashValue = hashlib.md5(line.rstrip().encode('utf-8')).hexdigest()

  if hashValue not in completed_lines_hash:
    output_file.write(line)
    completed_lines_hash.add(hashValue)

output_file.close()

In [ ]:
f = open("EN-FR.txt", "r")  #Load file in any mode that's able to read, ie r, r+, w+ etc
len(f.readlines())

149836

In [ ]:
#Hyperparameters
batch_size = 200     # Batch size for training.
latent_dim = 200      # Number of epochs to train for.
num_samples = 10000   # Number of samples to train on.

In [ ]:
#Vectorize the data.
input_texts = []
target_texts = []
input_chars = set()
target_chars = set()
with open('eng-french.txt', 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:  # Taking lines from 0 to num_samples which is 10000
    input_text, target_text = line.split('\t')    # We use "tab" as the "start sequence" character.
    target_text = '\t' + target_text + '\n'       # for the targets, and "\n" as "end sequence" character.
    #print(input_texts, target_texts) #pp
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_chars:
            input_chars.add(char)
    for char in target_text:
        if char not in target_chars:
            target_chars.add(char)

input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

#Print size
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)



Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [ ]:

#Define data for encoder and decoder
input_token_id = dict([(char, i) for i, char in enumerate(input_chars)])
target_token_id = dict([(char, i) for i, char in enumerate(target_chars)])

encoder_in_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

decoder_in_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):     #earlier we have made a numpy array of zeros
    for t, char in enumerate(input_text):                                         # so here we are one hot encoding by putting 1 on places chars are found.
        encoder_in_data[i, t, input_token_id[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_in_data[i, t, target_token_id[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_id[char]] = 1.

In [ ]:
#Define and process the input sequence
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
#We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

#Using `encoder_states` set up the decoder as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
#Final model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
#Model Summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 94)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 200), (None, 217600      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 200),  236000      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [ ]:
#Model data Shape
print("encoder_in_data shape:",encoder_in_data.shape)
print("decoder_in_data shape:",decoder_in_data.shape)
print("decoder_target_data shape:",decoder_target_data.shape)

encoder_in_data shape: (10000, 16, 71)
decoder_in_data shape: (10000, 59, 94)
decoder_target_data shape: (10000, 59, 94)


In [ ]:
from tensorflow.keras.optimizers import Adam
#Compiling and training the model
model.compile(optimizer=Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.001), loss='categorical_crossentropy')

model.fit([encoder_in_data, decoder_in_data], decoder_target_data, batch_size = batch_size, epochs=200, validation_split=0.3)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/200
35/35 [==============================] - 31s 788ms/step - loss: 1.0174 - val_loss: 1.1014
Epoch 2/200
35/35 [==============================] - 26s 756ms/step - loss: 0.9200 - val_loss: 1.0394
Epoch 3/200
35/35 [==============================] - 26s 749ms/step - loss: 0.8454 - val_loss: 0.9775
Epoch 4/200
35/35 [==============================] - 26s 732ms/step - loss: 0.7981 - val_loss: 0.9422
Epoch 5/200
35/35 [==============================] - 24s 691ms/step - loss: 0.7650 - val_loss: 0.9375
Epoch 6/200
35/35 [==============================] - 24s 692ms/step - loss: 0.7259 - val_loss: 0.9080
Epoch 7/200
35/35 [==============================] - 25s 720ms/step - loss: 0.7060 - val_loss: 0.8697
Epoch 8/200
35/35 [==============================] - 24s 679ms/step - loss: 0.6860 - val_loss: 0.8560
Epoch 9/200
35/35 [==============================] - 25s 717ms/step - loss: 0.6811 - val_loss: 0.8194
Epoch 10/200
35/35 [==============================] - 25s 703ms/step - loss: 0.642

In [ ]:
#Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [ ]:
reverse_input_char_index = dict((i, char) for char, i in input_token_id.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_id.items())

#Define Decode Sequence
def decode_sequence(input_seq):
    #Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    #Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #Get the first character of target sequence with the start character.
    target_seq[0, 0, target_token_id['\t']] = 1.

    #Sampling loop for a batch of sequences
    #(to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        #Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        #Exit condition: either hit max length
        #or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        #Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        #Update states
        states_value = [h, c]

    return decoded_sentence


In [ ]:
for seq_index in range(633,650,3):
    input_seq = encoder_in_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: I'm blind.
Decoded sentence: Je suis en train de mourir.

-
Input sentence: I'm clean.
Decoded sentence: Je suis pris de meis.

-
Input sentence: I'm cured.
Decoded sentence: Je suis contrarié.

-
Input sentence: I'm drunk.
Decoded sentence: Je suis continte.

-
Input sentence: I'm early.
Decoded sentence: Je suis prêt.

-
Input sentence: I'm fussy.
Decoded sentence: Je suis dur.

